# BP Experiments to verify theory for L-2 norm - CIFAR-10

In [1]:
%load_ext autoreload
%autoreload 2
from __future__ import division
from __future__ import print_function

import sys, os, gc, math
import numpy as np
from scipy.fftpack import dct,idct
from keras.datasets import cifar10
from PIL import Image
from multiprocessing import Pool
import time



sys.path.append('../')

from models.util import *


#Seed used for choosing classes, training points, and test points.
#SEED = 14
SEED=11

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
num_samples = 500
sqrt_n = 32
input_shape=(sqrt_n,sqrt_n)
n = sqrt_n*sqrt_n
k = 40
c=2.0

In [3]:
#Load Cifar10 data
(X_train, _), (X_test, _) = cifar10.load_data()
X_train = X_train.reshape(-1, 32, 32, 3)
X_test = X_test.reshape(-1, 32, 32, 3)

m_data = np.concatenate((X_train,X_test))

In [4]:
#Normalize the data
m_data = m_data/255.0

In [5]:
#Check cifar10 results for 1000 random images - BP
subset_idx = np.random.choice(np.arange(m_data.shape[0]),num_samples)
m_data_sub_bp = m_data[subset_idx]
m_data_y_bp = np.zeros((num_samples,sqrt_n,sqrt_n,3))
for i in range(num_samples):
    #first sample an element from the data
    x = m_data_sub_bp[i,:,:,:].flatten()
    x_hat = dct(x,norm='ortho')
    x_k = idct(get_topk_vec(x_hat,k=k),norm='ortho')
    e = np.random.uniform(size=n*3)
    y = x_k + e
    m_data_y_bp[i,:,:,:] = y.reshape((sqrt_n,sqrt_n,3))  
    m_data_sub_bp[i,:,:,:] = x_k.reshape((sqrt_n,sqrt_n,3))

In [6]:
#Form the matrix F
F = get_matrix(n,tf='dct')
n = 32*32

In [7]:
def approximate(z):
    y,eta = z
    x_hat = socp(y,F,n=n,eta=eta)
    return x_hat

In [ ]:
%%capture three
#BP CIFAR10
errors_l2_bp = np.zeros(m_data_y_bp.shape[0])
errors_l1_bp = np.zeros(m_data_y_bp.shape[0])
eta_bp = np.zeros((m_data_y_bp.shape[0],3))
diff_l2_bp = np.zeros(m_data_y_bp.shape[0])
diff_l1_bp = np.zeros(m_data_y_bp.shape[0])

for i in range(num_samples):
    y_r = m_data_y_bp[i,:,:,0].flatten()
    x_r = m_data_sub_bp[i,:,:,0].flatten()
    
    #Get actual top k and bottom k
    x_hat_top_k_r = dct(x_r, norm='ortho')
    
    e_r = y_r - x_r
    eta_r = eta_bp[i,0] = np.linalg.norm(e_r)
    
    y_g = m_data_y_bp[i,:,:,1].flatten()
    x_g = m_data_sub_bp[i,:,:,1].flatten()
    
    #Get actual top k and bottom k
    x_hat_top_k_g = dct(x_g, norm='ortho') 
    
    e_g = y_g - x_g
    eta_g = eta_bp[i,1] = np.linalg.norm(e_g)
    
    
    y_b = m_data_y_bp[i,:,:,2].flatten()
    x_b = m_data_sub_bp[i,:,:,2].flatten()
    
    #Get actual top k and bottom k
    x_hat_top_k_b =  dct(x_b, norm='ortho') 
    
    e_b = y_b - x_b
    eta_b = eta_bp[i,2] = np.linalg.norm(e_b)
    
    args = [(y_r,eta_r), (y_g,eta_g), (y_b,eta_b)]
    p = Pool(3)
    approximates = p.map(approximate, args)
    p.terminate()

    x_hat_approx_r = approximates[0]
    x_hat_approx_g = approximates[1]
    x_hat_approx_b = approximates[2]

    
    #Note the errors
    errors_l2_bp[i] = np.linalg.norm(x_hat_top_k_r.flatten()- x_hat_approx_r.flatten()) + \
                        np.linalg.norm(x_hat_top_k_g.flatten()- x_hat_approx_g.flatten()) + \
                            np.linalg.norm(x_hat_top_k_b.flatten()- x_hat_approx_b.flatten())
    errors_l1_bp[i] = np.linalg.norm(x_hat_top_k_r.flatten()- x_hat_approx_r.flatten(),ord=1) + \
                        np.linalg.norm(x_hat_top_k_g.flatten()- x_hat_approx_g.flatten(),ord=1) + \
                            np.linalg.norm(x_hat_top_k_b.flatten()- x_hat_approx_b.flatten(),ord=1)



    #Get the multiplicative constant
    c_l2_r = 6*eta_bp[i,0]
    c_l1_r = np.sqrt(k)*6*eta_bp[i,0]
    c_l2_g = 6*eta_bp[i,1]
    c_l1_g = np.sqrt(k)*6*eta_bp[i,1]
    c_l2_b = 6*eta_bp[i,2]
    c_l1_b = np.sqrt(k)*6*eta_bp[i,2]
    
    
    #Calculate the difference from the upper bound
    diff_l2_bp[i] = c_l2_r + c_l2_g + c_l2_b - errors_l2_bp[i] 
    diff_l1_bp[i] = c_l1_r + c_l1_g + c_l1_b - errors_l1_bp[i]

In [15]:
print(np.mean(errors_l2_bp), 
      np.mean(diff_l2_bp), 
      np.mean(errors_l1_bp), 
      np.mean(diff_l1_bp))

17.454773209090032 315.222592295217 88.5623598605005 2015.4740420953808


In [16]:
e_norms = np.zeros(m_data_y_bp.shape)
for i in range(num_samples):
    y = m_data_y_bp[i,:,:,:].flatten()
    x = m_data_sub_bp[i,:,:,:].flatten()
    e_norms[i] = np.linalg.norm(y - x)

In [17]:
np.mean(e_norms)

32.01397038302063

In [18]:
mnist_tup_bp = (m_data_y_bp, m_data_sub_bp, errors_l2_bp, errors_l1_bp, diff_l2_bp, diff_l1_bp )

import pickle
with open('data/cifar_theory_socp_l2.pickle', 'wb') as f:
    pickle.dump(mnist_tup_bp, f)
